In [1]:
!pip install -q datasets transformers pytorch-lightning wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
!wget https://samate.nist.gov/SARD/downloads/test-suites/2022-05-12-php-test-suite-sqli-v1-0-0.zip
!wget https://samate.nist.gov/SARD/downloads/test-suites/2015-10-27-php-vulnerability-test-suite.zip

--2023-10-16 10:34:41--  https://samate.nist.gov/SARD/downloads/test-suites/2022-05-12-php-test-suite-sqli-v1-0-0.zip
Resolving samate.nist.gov (samate.nist.gov)... 129.6.13.19, 2610:20:6005:13::19
Connecting to samate.nist.gov (samate.nist.gov)|129.6.13.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 976578941 (931M) [application/zip]
Saving to: ‘2022-05-12-php-test-suite-sqli-v1-0-0.zip’

2022-05-12-php-test 100%[===================>] 931.34M  36.3MB/s    in 26s     

2023-10-16 10:35:08 (35.7 MB/s) - ‘2022-05-12-php-test-suite-sqli-v1-0-0.zip’ saved [976578941/976578941]

--2023-10-16 10:35:09--  https://samate.nist.gov/SARD/downloads/test-suites/2015-10-27-php-vulnerability-test-suite.zip
Resolving samate.nist.gov (samate.nist.gov)... 129.6.13.19, 2610:20:6005:13::19
Connecting to samate.nist.gov (samate.nist.gov)|129.6.13.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167957206 (160M) [application/zip]
Saving to: ‘2015-10-

In [3]:
!mkdir dataset
!unzip -o -q 2022-05-12-php-test-suite-sqli-v1-0-0.zip -d dataset
!unzip -o -q 2015-10-27-php-vulnerability-test-suite.zip -d dataset

In [4]:
import os
import json
import re
import random

# Define the path to the dataset directory
dataset_dir = "dataset"

# Initialize a list to store the extracted information
results = {
    "directory_name": [],
    "code_snippet": [],
    "cwe_id": []
}

# Iterate through all directories in the dataset
l = os.listdir(dataset_dir)
random.shuffle(l)
l = l[:25000]
for directory in l:
    directory_path = os.path.join(dataset_dir, directory)
    if os.path.isdir(directory_path):
        # Check if there is a manifest.sariff file
        manifest_path = os.path.join(directory_path, "manifest.sarif")
        if os.path.exists(manifest_path):
            # Read the contents of the manifest file
            with open(manifest_path, "r") as f:
                manifest_data = json.load(f)

            # Extract relevant information from the manifest data
            code_snippet = None
            cwe_id = "None"

            for result in manifest_data["runs"][0]["results"]:
                code_location = result["locations"][0]["physicalLocation"]["artifactLocation"]["uri"]
                code_snippet = open(os.path.join(directory_path,code_location), "r").read()
                cwe_id = result["ruleId"]

                code_snippet = re.sub(r'<!--(.*?)-->', '', code_snippet, flags=re.DOTALL).strip('\n')

                results["directory_name"].append(directory)
                results["code_snippet"].append(code_snippet)
                results["cwe_id"].append(cwe_id)

In [5]:
labels = list(set(results["cwe_id"]))

In [6]:
from datasets import Dataset,DatasetDict,load_from_disk
dataset = Dataset.from_dict(results)

In [7]:
train_testvalid = dataset.train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['directory_name', 'code_snippet', 'cwe_id'],
        num_rows: 18048
    })
    test: Dataset({
        features: ['directory_name', 'code_snippet', 'cwe_id'],
        num_rows: 2257
    })
    validation: Dataset({
        features: ['directory_name', 'code_snippet', 'cwe_id'],
        num_rows: 2256
    })
})

In [9]:
dataset.save_to_disk('datasets')

Saving the dataset (0/1 shards):   0%|          | 0/18048 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2257 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2256 [00:00<?, ? examples/s]

In [10]:
!zip -r datasets.zip datasets

  adding: datasets/ (stored 0%)
  adding: datasets/train/ (stored 0%)
  adding: datasets/train/data-00000-of-00001.arrow (deflated 91%)
  adding: datasets/train/state.json (deflated 38%)
  adding: datasets/train/dataset_info.json (deflated 60%)
  adding: datasets/dataset_dict.json (deflated 2%)
  adding: datasets/test/ (stored 0%)
  adding: datasets/test/data-00000-of-00001.arrow (deflated 91%)
  adding: datasets/test/state.json (deflated 38%)
  adding: datasets/test/dataset_info.json (deflated 60%)
  adding: datasets/validation/ (stored 0%)
  adding: datasets/validation/data-00000-of-00001.arrow (deflated 91%)
  adding: datasets/validation/state.json (deflated 38%)
  adding: datasets/validation/dataset_info.json (deflated 60%)


In [11]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")

prefix = "Find the CWE ID: "
max_input_length = 512
max_target_length = 10

def preprocess_examples(examples):
    codes = examples['code_snippet']
    cwe_ids = examples['cwe_id']

    inputs = [prefix + code for code in codes]
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

    labels = tokenizer(cwe_ids, max_length=max_target_length, padding="max_length", truncation=True).input_ids

    labels = list(map(lambda x:x if x!=0 else -100,labels))

    model_inputs["labels"] = labels

    return model_inputs

In [12]:
dataset = dataset.map(preprocess_examples, batched=True)

Map:   0%|          | 0/18048 [00:00<?, ? examples/s]

Map:   0%|          | 0/2257 [00:00<?, ? examples/s]

Map:   0%|          | 0/2256 [00:00<?, ? examples/s]

In [13]:
from torch.utils.data import DataLoader

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [14]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [15]:
tokenizer.decode(batch['input_ids'][0])

'<s>Find the CWE ID: <?php\n# Init\n$db = new SQLite3("/var/www/db/database.db");\n\n\n# Sample\n$tainted = filter_input_array(INPUT_GET, ["t" => FILTER_SANITIZE_ADD_SLASHES]);\n$tainted = $tainted["t"];\n$sanitized = str_replace(["\\"", "\'", "<", ">"], "", $tainted);\n$dataflow = $sanitized;\n$context = (("SELECT * FROM users WHERE pin =". $dataflow). ";");\n$results = $db->query($context);\nwhile(($row = $results->fetchArray()))\n{\n  echo(htmlentities(print_r($row, true)));\n}\n\n?></s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [16]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'<s>CWE-89</s><pad><pad><pad><pad>'

In [17]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=15, warmup_steps=1000):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs

    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss

    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}

        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [18]:
import wandb

wandb.login(key='d1a805e1a99af277ae9db03679bb9438ae56da86')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [19]:
model = CodeT5()

In [20]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

wandb_logger = WandbLogger(name='codet5-finetune-code-vulnerabilty-25k', project='FYP')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints",
                  logger=wandb_logger,
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/utilities.py:72: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
wandb: Currently logged in as: lu513n. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
241.969   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [21]:
save_directory = "model" # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)
!zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/config.json (deflated 61%)
  adding: model/generation_config.json (deflated 34%)
  adding: model/pytorch_model.bin (deflated 7%)


In [ ]:
dataset = load_from_disk('datasets')

In [23]:
test_example = dataset['test'][2]
print(test_example)
print("Code:", test_example['code_snippet'])

{'directory_name': '499624-v1.0.0', 'code_snippet': '<?php\n# Init\n$info = ["Database" => "myDB", "UID" => "SA", "PWD" => "Msql12345678!"];\n$db = sqlsrv_connect("mssql", $info);\nif($db == false)\n{\n  die(print_r(sqlsrv_errors(), true));\n}\n\n\n# Sample\n$tainted = filter_input(INPUT_GET, "t", FILTER_SANITIZE_URL);\n$sanitized = md5($tainted, false);\n$dataflow = $sanitized;\n$context = (("SELECT * FROM users WHERE pin =" . $dataflow) . ";");\n$result = sqlsrv_query($db, $context);\nwhile(($row = sqlsrv_fetch_object($result)))\n{\n  echo(htmlentities(print_r($row, true)));\n}\n\n?>', 'cwe_id': 'CWE-89'}
Code: <?php
# Init
$info = ["Database" => "myDB", "UID" => "SA", "PWD" => "Msql12345678!"];
$db = sqlsrv_connect("mssql", $info);
if($db == false)
{
  die(print_r(sqlsrv_errors(), true));
}


# Sample
$tainted = filter_input(INPUT_GET, "t", FILTER_SANITIZE_URL);
$sanitized = md5($tainted, false);
$dataflow = $sanitized;
$context = (("SELECT * FROM users WHERE pin =" . $dataflow) . "

In [24]:

# prepare for the model
input_ids = tokenizer(test_example['code_snippet'], return_tensors='pt').input_ids
# generate
outputs = model.model.generate(input_ids)
print("Generated docstring:", tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated docstring: CWE-89


In [25]:
print("Ground truth:", test_example['cwe_id'])

Ground truth: CWE-89


In [26]:
test_dataset = dataset['test']

correct_predictions = 0

for example in test_dataset:
    # Generate docstring
    input_ids = tokenizer(example['code_snippet'], return_tensors='pt').input_ids
    outputs = model.model.generate(input_ids)
    generated_docstring = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Get the first word from generated docstring
    generated_first_word = generated_docstring.split()[0].lower()

    # Get the first word from ground truth docstring
    ground_truth_first_word = example['cwe_id'].split()[0].lower()

    # Check if the first word matches
    if generated_first_word == ground_truth_first_word:
        correct_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / len(test_dataset)

print(f"Accuracy: {accuracy}")

Token indices sequence length is longer than the specified maximum sequence length for this model (611 > 512). Running this sequence through the model will result in indexing errors


Accuracy: 0.9672131147540983
